In [ ]:
import cv2
from collections import Counter
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import sys
sys.path.append('../')
import TreeConverter as tc

In [ ]:
image = tc.Image('../test_data/dendrogram3.png', resize_factor=0.25)
v_lines, h_lines, internal_nodes, leaves = image.tree_image.find_lines_intersections_leaves(legend=True,orientation='horizontal',
                                                 intersection_threshold=5, min_freq=100)

In [ ]:
image = tc.Image('../test_data/tree4.png', resize_factor=0.25)
v_lines, h_lines, internal_nodes, leaves = image.tree_image.find_lines_intersections_leaves(legend=True,orientation='horizontal',
                                                 intersection_threshold=7, min_freq=100)

In [ ]:
image = tc.Image('../test_data/tree5.png', resize_factor=0.25)
v_lines, h_lines, internal_nodes, leaves = image.tree_image.find_lines_intersections_leaves(legend=True,orientation='horizontal',
                                                 intersection_threshold=15, min_freq=100)

In [ ]:
image = tc.Image('../test_data/tree6.png', resize_factor=0.25)
v_lines, h_lines, internal_nodes, leaves = image.tree_image.find_lines_intersections_leaves(legend=True,orientation='horizontal',
                                                 intersection_threshold=5, min_freq=100)

In [ ]:
image = tc.Image('../test_data/tree_a.png', resize_factor=0.25)
v_lines, h_lines, internal_nodes, leaves = image.tree_image.find_lines_intersections_leaves(legend=True,orientation='horizontal',
                                                 intersection_threshold=5, min_freq=100)

In [ ]:
image = tc.Image('../test_data/tree_b.png', resize_factor=0.25)
v_lines, h_lines, internal_nodes, leaves = image.tree_image.find_lines_intersections_leaves(legend=True,orientation='horizontal',
                                                 intersection_threshold=5, min_freq=100)

In [ ]:
image = tc.Image('../test_data/big_tree.png', resize_factor=0.5)
v_lines, h_lines, internal_nodes, leaves = image.tree_image.find_lines_intersections_leaves(legend=True,orientation='horizontal',
                                                 intersection_threshold=5, min_freq=100)

In [ ]:
tree_image = tc.TreeImage('../test_data/big_tree_tree_cropped.png', resize_factor=4)
tree_image.find_lines_intersections_leaves(legend=True, orientation='horizontal',
                                           intersection_threshold=10, min_freq=8)

### Trees

In [ ]:
image = tc.TreeImage('../test_data/dendogram2.png')
v_lines, h_lines, filtered_intersections, filtered_leaves = image.find_lines_intersections_leaves(legend=True,
                                                                                                 orientation="vertical",
                                                                                                 intersection_threshold=5)

In [ ]:
image = tc.TreeImage('../test_data/dendrogram_without_text.png')
v_lines, h_lines, filtered_intersections, filtered_leaves = image.find_lines_intersections_leaves(legend=True, 
                                                                                                 orientation="vertical")

In [ ]:
image = tc.TreeImage('../test_data/dendrogram_without_text_inverted.png')
v_lines, h_lines, filtered_intersections, filtered_leaves = image.find_lines_intersections_leaves(legend=True,
                                                                                                 orientation='horizontal')

In [ ]:
image = tc.TreeImage('../test_data/asia_tree1.png')
v_lines, h_lines, filtered_intersections, filtered_leaves = image.find_lines_intersections_leaves(legend=False,
                                                                                                 orientation='horizontal', 
                                                                                                 intersection_threshold=6)